### sklearn.linear_model.LogisticRegression
#### 종속변수가 범주형일 경우 적용 가능
#### 주요 Hyperparameter
- C : 기본값 1로, 작을수록 모델 단순, 클수록 모델 복잡. Logscale(0.01, 0.1, 1, 10 등) 단위로 최적값 탐색 권고
- solver : 데이터 양이 수 십만 이상일 경우 Full-Batch이면, 시간이 오래 걸리므로 'sag' (평균경사하강법)를 적용한 빠른 속도 적용 가능

#### LogisticRegression 옵션
##### LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)

# 분석 코드

In [1]:
# 라이브러리 및 데이터 로드
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import randint
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

df = pd.read_csv('../input/big-data-certification-study/breast-cancer-wisconsin.csv', encoding='utf-8')
df.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [2]:
# 데이터셋 분리
X = df.drop(columns=['code', 'Class'])
y = df[['Class']]

df.shape, X.shape, y.shape

((683, 11), (683, 9), (683, 1))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
# stratify : y 범주의 비율에 따라 데이터 분리

In [4]:
# 정규화 - Min-Max Scaling
scaler=MinMaxScaler()
scaler.fit(X_train) # feature data의 train data만 fit
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [5]:
# 모델 적용/예측 및 정확도
model=LogisticRegression()
model.fit(scaled_X_train, y_train)
pred_train = model.predict(scaled_X_train)
model.score(scaled_X_train, y_train)

0.97265625

In [6]:
# 혼동행렬 확인
con_train = confusion_matrix(y_train, pred_train)
print('Train Data 혼동행렬 :\n', con_train)

# 정상(0) 중 5명 오분류, 환자(1) 중 9명 오분류

Train Data 혼동행렬 :
 [[328   5]
 [  9 170]]


In [7]:
# 분류예측 레포트 확인
cfr_train = classification_report(y_train, pred_train)
print('Train Data 분류예측 레포트 :\n', cfr_train)

Train Data 분류예측 레포트 :
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       333
           1       0.97      0.95      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



In [8]:
# 모델 적용/예측 및 정확도
pred_test = model.predict(scaled_X_test)
model.score(scaled_X_test, y_test)

0.9590643274853801

In [9]:
# 혼동행렬 확인
con_test = confusion_matrix(y_test, pred_test)
print('Test Data 혼동행렬 :\n', con_test)

# 정상(0) 중 5명 오분류, 환자(1) 중 2명 오분류

Test Data 혼동행렬 :
 [[106   5]
 [  2  58]]


In [10]:
# 분류예측 레포트 확인
cfr_test = classification_report(y_test, pred_test)
print('Test Data 분류예측 레포트 :\n', cfr_test)

Test Data 분류예측 레포트 :
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [11]:
# Hyperparameter Tuning
# Grid Search
param_grid={'C':[0.001,0.01,0.1,1,10,100]}
grid_search=GridSearchCV(LogisticRegression(),
                         param_grid, cv=5,
                         return_train_score=True)
grid_search.fit(scaled_X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             return_train_score=True)

In [12]:
print('Best Parameter :', grid_search.best_params_)
print('Best Score :', round(grid_search.best_score_, 3))
print('Test Set Score :', round(grid_search.score(scaled_X_test, y_test), 3))

Best Parameter : {'C': 10}
Best Score : 0.973
Test Set Score : 0.959


In [13]:
# Randomized Search
param_r={'C':randint(low=0.001, high=100)}
r_search=RandomizedSearchCV(LogisticRegression(),
                            param_distributions=param_r,
                            cv=5, n_iter=100,
                            return_train_score=True)
r_search.fit(scaled_X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f114e17cb10>},
                   return_train_score=True)

In [14]:
print('Best Parameter :', r_search.best_params_)
print('Best Score :', round(r_search.best_score_, 3))
print('Test Set Score :', round(r_search.score(scaled_X_test, y_test), 3))

Best Parameter : {'C': 14}
Best Score : 0.975
Test Set Score : 0.959
